# Deep Learning - Exercise 11

This exercise focuses on time series forecasting using deep learning techniques. We will apply these methods to predict natural gas consumption, building upon the pre-processed dataset from previous exercises.

**Core Concepts**
* 📈 Time series forecasting with deep learning
* ⛽ Natural gas consumption prediction
* 📊 Utilizing pre-processed time series datasets
* 🧠 Implementing deep learning models for time series data
* 🛠️ Practical application of deep learning to real-world forecasting problems

The raw dataset is available at [vsb.ai](https://vsb.ai/natural-gas-forecasting), and we will be using a pre-processed version for this exercise.

[Open in Google colab](https://colab.research.google.com/github/rasvob/VSB-FEI-Deep-Learning-Exercises/blob/main/dl_11.ipynb)

[Download from Github](https://github.com/rasvob/VSB-FEI-Deep-Learning-Exercises/blob/main/dl_11.ipynb)

In [ ]:
!pip install tqdm
from tqdm.notebook import trange, tqdm
tqdm.pandas()

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
pd.set_option('display.max_colwidth', 100)
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, Input, LSTM
from tensorflow.keras.models import Model
import datetime
import random

tf.version.VERSION

In [ ]:
def reset_keras_session(seed=42):
    """
    Resets the Keras session and sets the random seeds for TensorFlow, NumPy, and Python.

    Args:
        seed (int): The seed to use for random number generation.
    """
    tf.keras.backend.clear_session()  # Clear the previous session.

    # Set seeds for reproducibility
    tf.random.set_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

    #Optional, but recommended for full reproducibility on some systems.
    #This also helps with GPU determinism.
    tf.config.experimental.reset_memory_stats('GPU:0') #if you have gpu.

    #Further optional, but can help with GPU determinism.
    # tf.config.experimental.enable_op_determinism()

    # If using CUDA, also set the following environment variable:
    # import os
    # os.environ['TF_DETERMINISTIC_OPS'] = '1'

# Example usage:
reset_keras_session(seed=13)

### We have prepared common metrics for model evaluation beforehand. These functions will be used later in the notebook to assess the performance of our models.

In [ ]:
"""
Computes MAPE
"""
def mean_absolute_percentage_error(y_true: np.array, y_pred: np.array) -> float:
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

"""
Computes SMAPE
"""
def symetric_mean_absolute_percentage_error(y_true: np.array, y_pred: np.array) -> float:
    return np.mean(np.abs((y_pred - y_true) / ((np.abs(y_true) + np.abs(y_pred))/2.0))) * 100

"""
Computes MAE, MSE, MAPE, SMAPE, R2
"""
def compute_metrics(df: pd.DataFrame) -> pd.DataFrame:
    y_true, y_pred = df['y_true'].values, df['y_pred'].values
    return compute_metrics_raw(y_true, y_pred)

def compute_metrics_raw(y_true: pd.Series, y_pred: pd.Series) -> pd.DataFrame:
    mae, mse, mape, smape, r2 = mean_absolute_error(y_true=y_true, y_pred=y_pred), mean_squared_error(y_true=y_true, y_pred=y_pred), mean_absolute_percentage_error(y_true=y_true, y_pred=y_pred), symetric_mean_absolute_percentage_error(y_true=y_true, y_pred=y_pred), r2_score(y_true=y_true, y_pred=y_pred)
    return pd.DataFrame.from_records([{'MAE': mae, 'MSE': mse, 'MAPE': mape, 'SMAPE': smape, 'R2': r2}], index=[0])

In [ ]:
def show_history(history):
    plt.figure()
    for key in history.history.keys():
        plt.plot(history.epoch, history.history[key], label=key)
    plt.legend()
    plt.tight_layout()

In [ ]:
df = pd.read_csv('https://github.com/rasvob/VSB-FEI-Deep-Learning-Exercises/raw/main/datasets/ppnet_metar_v8_MAD.csv', sep=';', index_col=0)
df = df[df.Year < 2019].copy()
df.loc[:, 'TestSet'] = 0
df.loc[df.Year == 2018, 'TestSet'] = 1
df = df.loc[:, ['Year', 'Month', 'Day', 'Hour', 'Day_of_week', 'Holiday', 'Consumption', 'Temperature', 'Pressure', 'Humidity', 'TestSet']]
df.index = pd.DatetimeIndex(df.index)
df.head()

# 📊 Dataset Description

This comprehensive dataset spans six complete years (January 1, 2013 to December 31, 2018), with all features recorded at hourly intervals, yielding 52,584 data points. The dataset integrates three essential components:

## 🏭 Consumption Data

The natural gas consumption data comes from Prague, the capital of the Czech Republic, with a distribution network serving 422,926 customers in 2018 and total consumption of 3.82 billion m³. The consumption is distributed across:

- **🏘️ Residential sector**: 381,914 households (33.3% of consumption)
- **🏢 Industrial sector**: 
  - 177 large customers (24.8%)
  - 39,175 medium customers (18.9%)
  - 1,652 small customers (21.9%)

The remaining percentage represents operational losses during distribution (e.g., pipeline leaks).

**🔥 Heating Season Information**:
- Period: September 1 to May 31
- Activation criteria: Temperature below +13°C for two consecutive days with no forecast warming
- Significance: Accounts for approximately 70-75% of annual consumption

## 🌡️ Weather Variables

Weather data was collected from the Prague LKPR airport weather station through METAR (aerodrome routine meteorological report) information. These standardized reports are regularly issued by airports and maintained in long-term archives, providing reliable meteorological data for our analysis.

## 💰 Economic Features

The economic component consists of natural gas price data obtained from the Czech energy regulation office. This data provides crucial context for understanding consumption patterns in relation to market conditions.

# Endogenous and Exogenous Variables in Forecasting Models

In time series forecasting, understanding the distinction between endogenous and exogenous variables is fundamental for building effective models. This classification helps determine the structure of your forecasting approach and influences how you interpret relationships between variables.

## Variable Classification

### Endogenous Variable
- **Consumption** 📊: This is your primary target variable being forecasted within the model system. It's considered endogenous because its values are determined within the model and influenced by other variables in the system as well as its own past values.

### Exogenous Variables
These variables are determined outside your forecasting model and serve as inputs:

- **Weather Variables** 🌡️: Data from the Prague LKPR airport weather station including temperature, humidity, precipitation, wind speed, and other meteorological measurements. These factors significantly influence energy consumption patterns but are not affected by consumption itself.

- **Economic Indicators** 💹: Natural gas price data from the Czech energy regulation office, which affects consumption behavior but is determined by broader market forces.

- **Temporal Features** 📅: Time-related variables such as hour of day, day of week, month, and seasonal indicators derived from timestamps. These cyclical patterns help capture regular consumption behaviors.


## We have ~ 52k datapoints which should be sufficient even for very complex models

In [ ]:
df.shape

In [ ]:
px.line(y=df['Consumption'], x=df.index, color=df.Year)

In [ ]:
px.line(y=df['Temperature'], x=df.index, color=df.Year)

# We will start with simple next step prediction using LSTM network
* We will limit the input to consumption variable first

## We will form input sequences first
* We will use 168 hours of past data
* Data will be scaled by `MinMaxScaler`

In [ ]:
def create_sequences(data, n_steps):
    """
    Create sequences of data for LSTM input
    data: input array
    n_steps: number of time steps (lags)
    """
    X, y = [], []
    for i in range(len(data) - n_steps):
        X.append(data[i:i + n_steps])
        y.append(data[i + n_steps])
    return np.array(X), np.array(y)

# Get the consumption data and convert to numpy array
consumption = df['Consumption'].values.reshape(-1, 1)

# Define the number of lags (time steps)
n_steps = 168  # Use 168 hours of data to predict the next hour

# Split data into train and test sets
train_data = consumption[df.TestSet == 0]
test_data = consumption[df.TestSet == 1]

# Scale the input data
scaler = MinMaxScaler(feature_range=(0, 1))
train_data_scaled = scaler.fit_transform(train_data)
test_data_scaled = scaler.transform(test_data)

# Create sequences for LSTM
X_train, y_train = create_sequences(train_data_scaled, n_steps)
X_test, y_test = create_sequences(test_data_scaled, n_steps)

# Scale the output data
* Reshape for LSTM [samples, time steps, features]

In [ ]:
# Reshape for LSTM [samples, time steps, features]
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

print(f"Training data shape: {X_train.shape}, {y_train.shape}")
print(f"Testing data shape: {X_test.shape}, {y_test.shape}")

# Check for np.nan or np.inf in the data
* If there are any, the model will fail to converge - you will get ``NaNs`` in the loss

In [ ]:
np.any(np.isnan(X_train)), np.any(np.isnan(y_train)), np.any(np.isnan(X_test)), np.any(np.isnan(y_test))

## Define the model

In [ ]:
inputs = keras.layers.Input(shape=(X_train.shape[1], X_train.shape[2]))
x = keras.layers.LSTM(128, activation='relu', return_sequences=True)(inputs)
# x = keras.layers.Dropout(0.2)(x)
# x = keras.layers.LSTM(64, activation='relu')(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dropout(0.2)(x)
outputs = keras.layers.Dense(1, activation='linear')(x)

model = keras.Model(inputs, outputs)

# Compile the model
model.compile(optimizer='adamW', loss='mse', metrics=['mae'])

model.summary()

In [ ]:
# Create callbacks
checkpoint = ModelCheckpoint(
    'best_lstm_model.h5',
    monitor='val_loss',
    save_best_only=True,
    mode='min',
)

# Train the model
history = model.fit(
    X_train, y_train,
    epochs=10,
    batch_size=32,
    validation_split=0.2,
    callbacks=[checkpoint],
)

# Plot the training history
show_history(history)

## Now we need to predict the test data and rescale it back to the original scale

In [ ]:
# Load the best model weights
model.load_weights('best_lstm_model.h5')

# Evaluate on test data
y_pred = model.predict(X_test).ravel()

# Scale the data back to original values
y_pred_rescaled = scaler.inverse_transform(y_pred.reshape(-1, 1)).ravel()
y_test_rescaled = scaler.inverse_transform(y_test.reshape(-1, 1)).ravel()

In [ ]:
# Create results dataframe for evaluation
results_df = pd.DataFrame({
    'y_true': y_test_rescaled,
    'y_pred': y_pred_rescaled
})

# Calculate metrics
metrics = compute_metrics(results_df)
metrics

## And there is the forecast visualized

In [ ]:
# Plot actual vs predicted values
fig = px.line(
    results_df,
    title='LSTM Model: Actual vs Predicted Consumption'
)
fig.update_layout(xaxis_title='Time', yaxis_title='Consumption')
fig.show()

## We can very easily test the CNN model as well
# TODO: Add the `dilation_rate` parameter description

In [ ]:
# Define the model
inputs = Input(shape=(X_train.shape[1], X_train.shape[2]))
x = Conv1D(filters=128, kernel_size=5, dilation_rate=2, activation='relu')(inputs)
x = MaxPooling1D(pool_size=2)(x)
x = Conv1D(filters=128, kernel_size=3, dilation_rate=2, activation='relu')(x)
x = MaxPooling1D(pool_size=2)(x)
x = Flatten()(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.2)(x)
outputs = Dense(1, activation='linear')(x)

model = Model(inputs, outputs)
model.compile(optimizer='adamW', loss='mse', metrics=['mae'])
model.summary()

In [ ]:
# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2, callbacks=[checkpoint])

# Plot the training history
show_history(history)

In [ ]:
# Evaluate on test data
y_pred = model.predict(X_test).ravel()

# Scale the data back to original values
y_pred_rescaled = scaler.inverse_transform(y_pred.reshape(-1, 1)).ravel()
y_test_rescaled = scaler.inverse_transform(y_test.reshape(-1, 1)).ravel()

# Create results dataframe for evaluation
results_df = pd.DataFrame({
    'y_true': y_test_rescaled,
    'y_pred': y_pred_rescaled
})

# Calculate metrics
metrics = compute_metrics(results_df)
metrics

In [ ]:
# Plot actual vs predicted values
fig = px.line(
    results_df,
    title='CNN Model with Dilated Convolution: Actual vs Predicted Consumption'
)
fig.update_layout(xaxis_title='Time', yaxis_title='Consumption')
fig.show()

## Now we can move to using other exogenous variables

In [ ]:
# Create sin and cos features for time components
def create_time_features(df):
    df['Month_sin'] = np.sin(2 * np.pi * df['Month'] / 12)
    df['Month_cos'] = np.cos(2 * np.pi * df['Month'] / 12)
    df['Day_sin'] = np.sin(2 * np.pi * df['Day'] / 31)
    df['Day_cos'] = np.cos(2 * np.pi * df['Day'] / 31)
    df['Hour_sin'] = np.sin(2 * np.pi * df['Hour'] / 24)
    df['Hour_cos'] = np.cos(2 * np.pi * df['Hour'] / 24)
    df['Day_of_week_sin'] = np.sin(2 * np.pi * df['Day_of_week'] / 7)
    df['Day_of_week_cos'] = np.cos(2 * np.pi * df['Day_of_week'] / 7)
    return df

df = create_time_features(df)
df.head()

In [ ]:
# Select the features to be used
features = ['Month_sin', 'Month_cos', 'Day_sin', 'Day_cos', 'Hour_sin', 'Hour_cos', 
           'Day_of_week_sin', 'Day_of_week_cos', 'Holiday', 'Consumption', 
           'Temperature', 'Pressure', 'Humidity']

# Create sequences for forecasting 24 hours ahead
def create_sequences(data, raw_data, n_steps):
    X, y = [], []
    for i in range(len(data) - n_steps):
        X.append(data[i:i + n_steps])
        y.append(raw_data[i + n_steps, features.index('Consumption')])
    return np.array(X), np.array(y)

In [ ]:
# Scale the input data
scaler = MinMaxScaler(feature_range=(0, 1))
train_data_scaled = scaler.fit_transform(df[features][df['TestSet'] == 0])
test_data_scaled = scaler.transform(df[features][df['TestSet'] == 1])

In [ ]:
# Create sequences for LSTM
n_steps = 168 
X_train, y_train = create_sequences(train_data_scaled, df[features][df['TestSet'] == 0].values, n_steps)
X_test, y_test = create_sequences(test_data_scaled, df[features][df['TestSet'] == 1].values, n_steps)

In [ ]:
# Scale the output data
output_scaler = MinMaxScaler(feature_range=(0, 1))
y_train_scaled = output_scaler.fit_transform(y_train.reshape(-1, 1))
y_test_scaled = output_scaler.transform(y_test.reshape(-1, 1))

In [ ]:
# Reshape for LSTM [samples, time steps, features]
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], len(features))
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], len(features))

print(f"Training data shape: {X_train.shape}, {y_train.shape}")
print(f"Testing data shape: {X_test.shape}, {y_test.shape}")

In [ ]:
inputs = keras.layers.Input(shape=(X_train.shape[1], X_train.shape[2]))
x = keras.layers.LSTM(128, activation='relu', return_sequences=True)(inputs)
# x = keras.layers.Dropout(0.2)(x)
# x = keras.layers.LSTM(64, activation='relu')(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dropout(0.2)(x)
outputs = keras.layers.Dense(1, activation='linear')(x)

model = keras.Model(inputs, outputs)

# Compile the model
model.compile(optimizer='adamW', loss='mse', metrics=['mae'])

model.summary()

In [ ]:
# Create callbacks
checkpoint = ModelCheckpoint(
    'best_lstm_model.h5',
    monitor='val_loss',
    save_best_only=True,
    mode='min',
)

# Train the model
history = model.fit(
    X_train, y_train_scaled,
    epochs=10,
    batch_size=32,
    validation_split=0.2,
    callbacks=[checkpoint],
)

# Plot the training history
show_history(history)

## Now we need to predict the test data and rescale it back to the original scale

In [ ]:
# Load the best model weights
model.load_weights('best_lstm_model.h5')

# Evaluate on test data
y_pred = model.predict(X_test).ravel()

# Scale the data back to original values
y_pred_rescaled = output_scaler.inverse_transform(y_pred.reshape(-1, 1)).ravel()
# y_test_rescaled = output_scaler.inverse_transform(y_test.reshape(-1, 1)).ravel()

In [ ]:
# Create results dataframe for evaluation
results_df = pd.DataFrame({
    'y_true': y_test,
    'y_pred': y_pred_rescaled
})

# Calculate metrics
metrics = compute_metrics(results_df)
metrics

## And there is the forecast visualized

In [ ]:
# Plot actual vs predicted values
fig = px.line(
    results_df,
    title='LSTM Model: Actual vs Predicted Consumption'
)
fig.update_layout(xaxis_title='Time', yaxis_title='Consumption')
fig.show()

## We need to form the next 24 hours forecast input and output sequences

In [ ]:
# Select the features to be used
features = ['Month_sin', 'Month_cos', 'Day_sin', 'Day_cos', 'Hour_sin', 'Hour_cos', 
           'Day_of_week_sin', 'Day_of_week_cos', 'Holiday', 'Consumption', 
           'Temperature', 'Pressure', 'Humidity']

# Create sequences for forecasting 24 hours ahead
def create_sequences(data, raw_data, n_steps):
    X, y = [], []
    for i in range(len(data) - n_steps - 24 + 1):
        X.append(data[i:i + n_steps])
        y.append(raw_data[i + n_steps:i + n_steps + 24, features.index('Consumption')])
    return np.array(X), np.array(y)

In [ ]:
# Create sin and cos features for time components
def create_time_features(df):
    df['Month_sin'] = np.sin(2 * np.pi * df['Month'] / 12)
    df['Month_cos'] = np.cos(2 * np.pi * df['Month'] / 12)
    df['Day_sin'] = np.sin(2 * np.pi * df['Day'] / 31)
    df['Day_cos'] = np.cos(2 * np.pi * df['Day'] / 31)
    df['Hour_sin'] = np.sin(2 * np.pi * df['Hour'] / 24)
    df['Hour_cos'] = np.cos(2 * np.pi * df['Hour'] / 24)
    df['Day_of_week_sin'] = np.sin(2 * np.pi * df['Day_of_week'] / 7)
    df['Day_of_week_cos'] = np.cos(2 * np.pi * df['Day_of_week'] / 7)
    return df

df = create_time_features(df)
df

In [ ]:
# Scale the input data
scaler = MinMaxScaler(feature_range=(0, 1))
train_data_scaled = scaler.fit_transform(df[features][df['TestSet'] == 0])
test_data_scaled = scaler.transform(df[features][df['TestSet'] == 1])

In [ ]:
# Create sequences for LSTM
n_steps = 168 
X_train, y_train = create_sequences(train_data_scaled, df[features][df['TestSet'] == 0].values, n_steps)
X_test, y_test = create_sequences(test_data_scaled, df[features][df['TestSet'] == 1].values, n_steps)

In [ ]:
# Scale the output data
output_scaler = MinMaxScaler(feature_range=(0, 1))
y_train_scaled = output_scaler.fit_transform(y_train.reshape(-1, 1)).reshape(-1, 24)
y_test_scaled = output_scaler.transform(y_test.reshape(-1, 1)).reshape(-1, 24)

In [ ]:
# Reshape for LSTM [samples, time steps, features]
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], len(features))
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], len(features))

print(f"Training data shape: {X_train.shape}, {y_train.shape}")
print(f"Testing data shape: {X_test.shape}, {y_test.shape}")

In [ ]:
# Build a model for 24-hour forecasting
inputs = Input(shape=(X_train.shape[1], X_train.shape[2]))
x = keras.layers.LSTM(128, activation='relu', return_sequences=True)(inputs)
# x = keras.layers.Dropout(0.2)(x)
# x = keras.layers.LSTM(64, activation='relu')(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dropout(0.2)(x)
outputs = Dense(24, activation='linear')(x)  # Output 24 values

model = keras.Model(inputs, outputs)

# Compile the model
model.compile(optimizer='adamW', loss='mse', metrics=['mse'])
model.summary()

In [ ]:
# Create callbacks
checkpoint = ModelCheckpoint(
    'best_forecast_model.h5',
    monitor='val_loss',
    save_best_only=True,
    mode='min',
)

# Train the model
history = model.fit(
    X_train, y_train_scaled,
    epochs=10,
    batch_size=32,
    validation_split=0.2,
    callbacks=[checkpoint]
)

# Plot the training history
show_history(history)

In [ ]:
# Make predictions
y_pred = model.predict(X_test)

# Inverse the scaling
y_pred_inv = output_scaler.inverse_transform(y_pred)

In [ ]:
y_pred_inv = output_scaler.inverse_transform(y_pred)

In [ ]:
midnight_indices = [i for i in range(len(y_test)) if df[df.TestSet == 1].iloc[i + n_steps].Hour == 0]
y_true_midnight = y_test[midnight_indices].ravel()
y_pred_midnight = y_pred_inv[midnight_indices].ravel()

In [ ]:
# Create results dataframe for evaluation
results_df = pd.DataFrame({
    'y_true': y_true_midnight,
    'y_pred': y_pred_midnight
})

# Calculate metrics
metrics = compute_metrics(results_df)
metrics

In [ ]:
# Plot actual vs predicted values
fig = px.line(
    results_df,
    title='Actual vs Predicted Consumption'
)
fig.update_layout(xaxis_title='Time', yaxis_title='Consumption')
fig.show()

# We can repeat the same process for the CNN model

In [ ]:
# Build a model for 24-hour forecasting
inputs = Input(shape=(X_train.shape[1], X_train.shape[2]))
x = Conv1D(filters=128, kernel_size=5, dilation_rate=2, activation='relu')(inputs)
x = MaxPooling1D(pool_size=2)(x)
x = Conv1D(filters=128, kernel_size=3, dilation_rate=2, activation='relu')(x)
x = MaxPooling1D(pool_size=2)(x)
x = Flatten()(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.2)(x)
outputs = Dense(24, activation='linear')(x)  # Output 24 values

model = keras.Model(inputs, outputs)

# Compile the model
model.compile(optimizer='adamW', loss='mse', metrics=['mse'])
model.summary()

In [ ]:
# Create callbacks
checkpoint = ModelCheckpoint(
    'best_forecast_model.h5',
    monitor='val_loss',
    save_best_only=True,
    mode='min',
)

# Train the model
history = model.fit(
    X_train, y_train_scaled,
    epochs=10,
    batch_size=32,
    validation_split=0.2,
    callbacks=[checkpoint]
)

# Plot the training history
show_history(history)

In [ ]:
# Make predictions
y_pred = model.predict(X_test)

# Inverse the scaling
y_pred_inv = output_scaler.inverse_transform(y_pred)

In [ ]:
y_pred_inv = output_scaler.inverse_transform(y_pred)

In [ ]:
midnight_indices = [i for i in range(len(y_test)) if df[df.TestSet == 1].iloc[i + n_steps].Hour == 0]
y_true_midnight = y_test[midnight_indices].ravel()
y_pred_midnight = y_pred_inv[midnight_indices].ravel()

In [ ]:
# Create results dataframe for evaluation
results_df = pd.DataFrame({
    'y_true': y_true_midnight,
    'y_pred': y_pred_midnight
})

# Calculate metrics
metrics = compute_metrics(results_df)
metrics

In [ ]:
# Plot actual vs predicted values
fig = px.line(
    results_df,
    title='Actual vs Predicted Consumption'
)
fig.update_layout(xaxis_title='Time', yaxis_title='Consumption')
fig.show()

## Transformer model

In [ ]:
# Select the features to be used
features = ['Month_sin', 'Month_cos', 'Day_sin', 'Day_cos', 'Hour_sin', 'Hour_cos', 
           'Day_of_week_sin', 'Day_of_week_cos', 'Holiday', 'Consumption', 
           'Temperature', 'Pressure', 'Humidity']

# Create sequences for forecasting 24 hours ahead
def create_sequences(data, raw_data, n_steps):
    X, y = [], []
    for i in range(len(data) - n_steps):
        X.append(data[i:i + n_steps])
        y.append(raw_data[i + n_steps, features.index('Consumption')])
    return np.array(X), np.array(y)

In [ ]:
# Scale the input data
scaler = MinMaxScaler(feature_range=(0, 1))
train_data_scaled = scaler.fit_transform(df[features][df['TestSet'] == 0])
test_data_scaled = scaler.transform(df[features][df['TestSet'] == 1])

In [ ]:
# Create sequences for LSTM
n_steps = 168 
X_train, y_train = create_sequences(train_data_scaled, df[features][df['TestSet'] == 0].values, n_steps)
X_test, y_test = create_sequences(test_data_scaled, df[features][df['TestSet'] == 1].values, n_steps)

In [ ]:
# Scale the output data
output_scaler = MinMaxScaler(feature_range=(0, 1))
y_train_scaled = output_scaler.fit_transform(y_train.reshape(-1, 1))
y_test_scaled = output_scaler.transform(y_test.reshape(-1, 1))

In [ ]:
# Reshape for LSTM [samples, time steps, features]
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], len(features))
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], len(features))

print(f"Training data shape: {X_train.shape}, {y_train.shape}")
print(f"Testing data shape: {X_test.shape}, {y_test.shape}")

In [ ]:
# Transformer Block
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Multi-head self attention
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    attention_output = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(x, x)
    attention_output = layers.Dropout(dropout)(attention_output)
    x = layers.Add()([inputs, attention_output])
    
    # Feed-forward network
    ff = layers.LayerNormalization(epsilon=1e-6)(x)
    ff = layers.Dense(ff_dim, activation="relu")(ff)
    ff = layers.Dropout(dropout)(ff)
    ff = layers.Dense(inputs.shape[-1])(ff)
    return layers.Add()([x, ff])

# Positional encoding
class PositionalEncoding(layers.Layer):
    def __init__(self, position, d_model):
        super().__init__()
        self.position = position
        self.d_model = d_model
        self.pos_encoding = self.positional_encoding(position, d_model)
    
    def get_angles(self, position, i, d_model):
        angles = 1 / tf.pow(10000, (2 * (i // 2)) / tf.cast(d_model, tf.float32))
        return position * angles
    
    def positional_encoding(self, position, d_model):
        angle_rads = self.get_angles(
            position=tf.cast(tf.range(position)[:, tf.newaxis], tf.float32),
            i=tf.cast(tf.range(d_model)[tf.newaxis, :], tf.float32),
            d_model=d_model
        )
        
        # Apply sin to even indices
        sines = tf.math.sin(angle_rads[:, 0::2])
        # Apply cos to odd indices
        cosines = tf.math.cos(angle_rads[:, 1::2])
        
        pos_encoding = tf.concat([sines, cosines], axis=-1)
        pos_encoding = pos_encoding[tf.newaxis, ...]
        
        return tf.cast(pos_encoding, tf.float32)
    
    def call(self, inputs):
        return inputs + self.pos_encoding[:, :tf.shape(inputs)[1], :]

In [ ]:
# Get the dimensions from your data
seq_length = X_train.shape[1]  # time steps
feature_dim = X_train.shape[2]  # number of features

# Define hyperparameters
head_size = 256
num_heads = 4
ff_dim = 4 * head_size
dropout_rate = 0.1
num_transformer_blocks = 2

In [ ]:
inputs = layers.Input(shape=(seq_length, feature_dim))

# Add positional encoding
x = PositionalEncoding(position=seq_length, d_model=feature_dim)(inputs)

# Transformer blocks
for _ in range(num_transformer_blocks):
    x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout_rate)

# Global average pooling
x = layers.GlobalAveragePooling1D()(x)

# Final prediction layer
outputs = layers.Dense(1, activation='linear')(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.AdamW(),
    loss='mse',
    metrics=['mae']
)

model.summary()

In [ ]:
# Create callbacks
checkpoint = ModelCheckpoint(
    'best_forecast_model.h5',
    monitor='val_loss',
    save_best_only=True,
    mode='min',
)

# Train the model
history = model.fit(
    X_train, y_train_scaled,
    epochs=10,
    batch_size=32,
    validation_split=0.2,
    callbacks=[checkpoint]
)

# Plot the training history
show_history(history)

In [ ]:
# Load the best model weights
model.load_weights('best_forecast_model.h5')

# Evaluate on test data
y_pred = model.predict(X_test).ravel()

# Scale the data back to original values
y_pred_rescaled = output_scaler.inverse_transform(y_pred.reshape(-1, 1)).ravel()
# y_test_rescaled = output_scaler.inverse_transform(y_test.reshape(-1, 1)).ravel()

In [ ]:
# Create results dataframe for evaluation
results_df = pd.DataFrame({
    'y_true': y_test,
    'y_pred': y_pred_rescaled
})

# Calculate metrics
metrics = compute_metrics(results_df)
metrics

## And there is the forecast visualized

In [ ]:
# Plot actual vs predicted values
fig = px.line(
    results_df,
    title='LSTM Model: Actual vs Predicted Consumption'
)
fig.update_layout(xaxis_title='Time', yaxis_title='Consumption')
fig.show()

# ✅  Tasks for the lecture (2p)

* Try to modify any of the proposed models or create a new one from scratch to get a lower forecasting error **(1p)**
    * Compare the new model with the original one. Did the MAE, MSE etc changed? If it did, how (is it better or worse, do you have any idea why)?